In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import shuffle
from scipy.stats import norm

sns.set(color_codes=True)

In [2]:
import_file_name = './scrapped_db.buildings.csv'
export_file_name = './keiv_lviv_toronto.csv'

In [3]:
df = pd.read_csv(import_file_name)
df.head()

,_id,addr:housenumber,addr:street,air_quality,amenity,area,bbox_id,building,building_centroid.coordinates.0,building_centroid.coordinates.1,...,humidity,name,request_area_bottom_left.coordinates.0,request_area_bottom_left.coordinates.1,request_area_bottom_left.type,request_area_top_right.coordinates.0,request_area_top_right.coordinates.1,request_area_top_right.type,shops_amount,temperature
0,5bd44a27aef2f0451bff5801,3,Тарасівська вулиця,71,NaN,1044.380159,Kyiv1,apartments,30.508372,50.439956,...,0.73,NaN,30.506201,50.432388,Point,30.520749,50.440698,Point,15.0,8.98
1,5bd44a27aef2f0451bff5802,22,Велика Васильківська вулиця,62,NaN,965.106000,Kyiv1,NaN,30.516853,50.439697,...,0.73,NaN,30.506201,50.432388,Point,30.520749,50.440698,Point,53.0,8.91
2,5bd44a27aef2f0451bff5803,20,Велика Васильківська вулиця,62,NaN,416.728161,Kyiv1,NaN,30.517242,50.439814,...,0.73,NaN,30.506201,50.432388,Point,30.520749,50.440698,Point,49.0,8.91
3,5bd44a27aef2f0451bff5804,45/2,Пушкінська вулиця,62,NaN,941.333197,Kyiv1,NaN,30.516350,50.439850,...,0.73,NaN,30.506201,50.432388,Point,30.520749,50.440698,Point,51.0,8.91
4,5bd44a27aef2f0451bff5805,9,Льва Толстого вулиця,62,university,591.428738,Kyiv1,NaN,30.512359,50.439901,...,0.73,Київський медичний університет,30.506201,50.432388,Point,30.520749,50.440698,Point,26.0,8.91


In [4]:
df.describe()

,air_quality,area,building_centroid.coordinates.0,building_centroid.coordinates.1,closest_shop,cloud_cover,geometry.coordinates.0.0.0,geometry.coordinates.0.0.1,geometry.coordinates.0.1.0,geometry.coordinates.0.1.1,...,geometry.coordinates.0.9.0,geometry.coordinates.0.9.1,height,humidity,request_area_bottom_left.coordinates.0,request_area_bottom_left.coordinates.1,request_area_top_right.coordinates.0,request_area_top_right.coordinates.1,shops_amount,temperature
count,16255.000000,16255.000000,16255.000000,16255.000000,16255.000000,16255.000000,16255.000000,16255.000000,16255.000000,16255.000000,...,3369.000000,3369.000000,16255.000000,16255.000000,16255.000000,16255.000000,16255.000000,16255.000000,15903.000000,16255.000000
mean,73.262504,708.635653,2.347983,48.621702,281.690188,0.641273,2.347910,48.621743,2.347967,48.621722,...,-5.892085,48.129132,15.961612,0.738576,2.337730,48.616721,2.358343,48.626345,20.321134,8.993379
std,6.191784,1866.035371,45.012884,2.733836,299.303792,0.083580,45.012893,2.733827,45.012891,2.733833,...,49.418342,3.000980,10.399679,0.012962,45.013477,2.734405,45.011066,2.733050,34.224472,0.783182
min,62.000000,0.010533,-79.754753,43.635565,0.000000,0.530000,-79.755058,43.635583,-79.755032,43.635522,...,-79.754729,43.636189,0.000000,0.720000,-79.764347,43.635578,-79.730358,43.651603,0.000000,8.010000
25%,69.000000,121.975725,23.964461,49.792113,64.000000,0.540000,23.964464,49.792096,23.964505,49.792146,...,-79.378379,43.722121,15.000000,0.730000,23.959701,49.787748,23.967834,49.800048,1.000000,8.370000
50%,74.000000,282.034578,24.040150,49.843943,183.000000,0.650000,24.040014,49.843936,24.039989,49.843935,...,24.031368,49.841954,15.000000,0.740000,24.036884,49.842411,24.044051,49.848554,5.000000,8.720000
75%,74.000000,661.699097,30.503865,50.434291,407.000000,0.730000,30.503791,50.434301,30.503830,50.434295,...,30.504055,50.433822,15.000000,0.750000,30.489120,50.426237,30.512252,50.439195,20.000000,9.160000
max,85.000000,82550.695498,30.646496,50.518033,2845.000000,0.730000,30.646597,50.518477,30.646488,50.519302,...,30.646474,50.517860,292.000000,0.770000,30.614004,50.507559,30.646448,50.517411,171.000000,10.560000


In [5]:
kiev_df = df.loc[df["bbox_id"].str.startswith("Kyiv")]
lviv_df = df.loc[df["bbox_id"].str.startswith("Lviv")]
toronto_df = df.loc[df["bbox_id"].str.startswith("Toronto")]

In [6]:
print("Number of buildings in Kiev: {}".format(kiev_df['bbox_id'].count()))
print("Number of buildings in Lviv: {}".format(lviv_df['bbox_id'].count()))
print("Number of buildings in Toronto: {}".format(toronto_df['bbox_id'].count()))

Number of buildings in Kiev: 5776
Number of buildings in Lviv: 6710
Number of buildings in Toronto: 3769


In [7]:
# select random data from dataframe

result_df = pd.DataFrame()

limit = 100

dfs = [kiev_df, lviv_df, toronto_df]
for df in dfs:
    ids = df['bbox_id'].unique()
    for curr_id in ids:
        part = df[df['bbox_id'] == curr_id].copy()
        result_df = result_df.append(shuffle(part).head(limit))
        
result_df.head()

,_id,addr:housenumber,addr:street,air_quality,amenity,area,bbox_id,building,building_centroid.coordinates.0,building_centroid.coordinates.1,...,humidity,name,request_area_bottom_left.coordinates.0,request_area_bottom_left.coordinates.1,request_area_bottom_left.type,request_area_top_right.coordinates.0,request_area_top_right.coordinates.1,request_area_top_right.type,shops_amount,temperature
140,5bd44a27aef2f0451bff588d,41,NaN,62,NaN,539.503101,Kyiv1,NaN,30.516521,50.436665,...,0.73,NaN,30.506201,50.432388,Point,30.520749,50.440698,Point,73.0,8.91
83,5bd44a27aef2f0451bff5854,8б,Антоновича вулиця,62,NaN,388.563045,Kyiv1,NaN,30.511799,50.438659,...,0.73,NaN,30.506201,50.432388,Point,30.520749,50.440698,Point,37.0,8.91
20,5bd44a27aef2f0451bff5815,1а,Тарасівська вулиця,71,NaN,285.506162,Kyiv1,NaN,30.508948,50.439938,...,0.73,NaN,30.506201,50.432388,Point,30.520749,50.440698,Point,15.0,8.98
204,5bd44a27aef2f0451bff58cd,17б,Антоновича вулиця,62,NaN,167.594028,Kyiv1,NaN,30.514814,50.437436,...,0.73,NaN,30.506201,50.432388,Point,30.520749,50.440698,Point,61.0,8.91
66,5bd44a27aef2f0451bff5843,NaN,Володимирська вулиця,71,NaN,1578.963971,Kyiv1,NaN,30.509057,50.438221,...,0.73,Корпус «Ж» НУХТ,30.506201,50.432388,Point,30.520749,50.440698,Point,27.0,8.98


In [8]:
# check that data are uniformly destributed by `bbox_id`
for bbox_id in result_df['bbox_id'].unique():
    print('Count of buildings in {}: {}'.format(bbox_id, result_df[result_df['bbox_id'] == bbox_id]['_id'].count()))

Count of buildings in Kyiv1: 100
Count of buildings in Kyiv2: 100
Count of buildings in Kyiv3: 100
Count of buildings in Kyiv4: 100
Count of buildings in Kyiv5: 100
Count of buildings in Kyiv6: 100
Count of buildings in Kyiv7: 100
Count of buildings in Kyiv8: 100
Count of buildings in Kyiv9: 100
Count of buildings in Kyiv10: 100
Count of buildings in Lviv1: 100
Count of buildings in Lviv2: 100
Count of buildings in Lviv3: 100
Count of buildings in Lviv4: 100
Count of buildings in Lviv5: 100
Count of buildings in Lviv6: 100
Count of buildings in Lviv7: 100
Count of buildings in Lviv8: 100
Count of buildings in Lviv9: 100
Count of buildings in Lviv10: 100
Count of buildings in Toronto1: 45
Count of buildings in Toronto2: 100
Count of buildings in Toronto3: 100
Count of buildings in Toronto4: 100
Count of buildings in Toronto5: 100
Count of buildings in Toronto6: 100
Count of buildings in Toronto7: 100
Count of buildings in Toronto8: 100
Count of buildings in Toronto9: 100
Count of buildi

In [9]:
result_df.to_csv(export_file_name)